In [1]:
# import the necessary packages
import numpy as np
import random
import time
import cv2
import os

In [2]:
# load the COCO class labels our Mask R-CNN was trained on
labelsPath = r"E:\AI Practice Project\cv\basic project\11-mask_rcnn\object_detection_classes_coco.txt"
LABELS = open(labelsPath).read().strip().split("\n")

# generate random color
'''colorsPath = r"E:\AI Practice Project\cv\basic project\11-mask_rcnn\colors.txt"
COLORS = open(colorsPath).read().strip().split("\n")
COLORS = [np.array(c.split(",")).astype("int") for c in COLORS]
COLORS = np.array(COLORS, dtype="uint8")
COLORS'''

colors = np.random.randint(0, 255, (90, 3))


In [3]:
# load mask-rcnn
weightsPath = r"E:\AI Practice Project\cv\basic project\11-mask_rcnn\frozen_inference_graph.pb"
configPath = r"E:\AI Practice Project\cv\basic project\11-mask_rcnn\mask_rcnn_inception_v2_coco_2018_01_28.pbtxt"
                               
net = cv2.dnn.readNetFromTensorflow(weightsPath, 
                                    configPath)

In [4]:
image_path = r"E:\AI Practice Project\cv\basic project\11-mask_rcnn\bike.jpg"
image = cv2.imread(image_path)
H, W = image.shape[:2]

blob = cv2.dnn.blobFromImage(image, swapRB=True, crop=False)
net.setInput(blob)
start = time.time()
# we need to access the last layers where we have the information
(boxes, masks) = net.forward(["detection_out_final", "detection_masks"])
end = time.time()

# show timing information and volume information on Mask R-CNN
print("[INFO] Mask R-CNN took {:.6f} seconds".format(end - start))
print("[INFO] boxes shape: {}".format(boxes.shape))
print("[INFO] masks shape: {}".format(masks.shape))

[INFO] Mask R-CNN took 3.057104 seconds
[INFO] boxes shape: (1, 1, 100, 7)
[INFO] masks shape: (100, 90, 15, 15)


In [7]:
# loop over the number of detected objects
for i in range(0, boxes.shape[2]):
    
    box = boxes[0,0,i]
    classID = int(box[1])
    confidence = box[2]

    if confidence > 0.7:
        # clone our original image so we can draw on it
        clone = image.copy()

        # scale the bounding box coordinates back relative to the
        # size of the image and then compute the width and the height of the bounding box
        box = boxes[0, 0, i, 3:7] * np.array([W, H, W, H])
        (startX, startY, endX, endY) = box.astype("int")
        boxW = endX - startX
        boxH = endY - startY

        # mask (value of each pixel)
        roi = image[startY:endY, startX:endX]
        roi_height, roi_width = roi.shape[:2]

        mask = masks[i, classID]
        mask = cv2.resize(mask, (roi_height, roi_width))
        _, mask = cv2.threshold(mask, 0.5, 255, cv2.THRESH_BINARY)

        # get mask cordinates
        contours, _ = cv2.findContours(np.array(mask, np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        color = colors[classID]
        for cnt in contours:
            cv2.fillPoly(roi, [cnt], (int(color[0]),int(color[1]),int(color[2])))

        cv2.rectangle(image, (startX, startY), (endX, endY), (0,255,100), 2)

        #mask = cv2.resize(mask, (boxW, boxH), interpolation=cv2.INTER_NEAREST)
        #mask = (mask > 0.3) 

#cv2.imshow('image', roi)
cv2.imshow('mask', image)
cv2.waitKey(0)

-1